Setting up the environment


In [ ]:
import os
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import rasterio
import torch
from torch.utils.data import Dataset,random_split,DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as T

# Importing the U-Net model
from models import Unet

Image transformations

In [ ]:
s1_transform=T.Compose([
    T.RandomHorizontalFlip(p=1),
    T.RandomVerticalFlip(p=0.5),
])

Data preparation

In [ ]:
class Sentinel1Dataset(Dataset):
    def __init__(self, chip_info_file, transform=None, target_transform=None):
        self.chip_df = pd.read_csv(chip_info_file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.chip_df)

    def __getitem__(self, idx):
        img_vv_path = self.chip_df.iloc[idx, 0]        
        img_vh_path = self.chip_df.iloc[idx, 1]
        dem_path = self.chip_df.iloc[idx, 2]
        pwat_path = self.chip_df.iloc[idx, 3]
        
        with rasterio.open(img_vv_path) as src_vv:
            img_vv = src_vv.read(1)
            img_vv = np.clip(img_vv,-50,25)
        with rasterio.open(img_vh_path) as src_vh:
            img_vh = src_vh.read(1)
            img_vh = np.clip(img_vh,-50,25)
        with rasterio.open(dem_path) as elevation:
            ele = elevation.read(1)
            ele = np.clip(ele,0.0,2000.0)
        with rasterio.open(pwat_path) as pwater:
            wat = pwater.read(1)
            wat = np.clip(wat,0.0,100.0)            
            
        # Normalization        
        img_vv_norm = (img_vv - (-50)) / (25 - (-50))
        img_vh_norm = (img_vh - (-50)) / (25 - (-50))
        ele_norm = ele/2000.0
        wat_norm = wat/100.0
        
        image = torch.stack((torch.from_numpy(img_vv_norm),
                             torch.from_numpy(img_vh_norm),
                             torch.from_numpy(ele_norm),
                             torch.from_numpy(wat_norm),))
            
        label_path = self.chip_df.iloc[idx, 4]
        with rasterio.open(label_path) as src_label:
            label = src_label.read(1)
        label = torch.from_numpy(label).to(torch.float32)
        label = torch.unsqueeze(label, 0)
    
        if self.transform:
            image = self.transform(image)
        
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
s1_data_set1 = Sentinel1Dataset('data/chip_set1.csv')
s1_data_set2 = Sentinel1Dataset('data/chip_set2.csv')

In [ ]:
s1_data_set1_T = Sentinel1Dataset('data/chip_set1.csv', transform=s1_transform, target_transform=s1_transform)
s1_data_set2_T = Sentinel1Dataset('data/chip_set2.csv', transform=s1_transform, target_transform=s1_transform)

In [ ]:
s1_data_set1 = torch.utils.data.ConcatDataset((s1_data_set1,s1_data_set1_T))
s1_data_set2 = torch.utils.data.ConcatDataset((s1_data_set2,s1_data_set2_T))

Splitting dataset into training and validation


In [ ]:
generator1 = torch.Generator().manual_seed(42)
generator2 = torch.Generator().manual_seed(42)

s1_set1_train, s1_set1_valid  = random_split(s1_data_set1, [0.9,0.1], generator=generator1)
s1_set2_train, s1_set2_valid  = random_split(s1_data_set2, [0.9,0.1], generator=generator2)

In [ ]:
#dataset1
print('train:',len(s1_set1_train),'validation:',len(s1_set1_valid))

In [ ]:
#dataset2
print('train:',len(s1_set2_train),'validation:',len(s1_set2_valid))

In [ ]:
train_set1_dataloader = DataLoader(s1_set1_train, batch_size=32, shuffle=True, pin_memory=True)
validation_set1_dataloader = DataLoader(s1_set1_valid, batch_size=32, shuffle=False, pin_memory=True)

train_set2_dataloader = DataLoader(s1_set2_train, batch_size=32, shuffle=True, pin_memory=True)
validation_set2_dataloader = DataLoader(s1_set2_valid, batch_size=32, shuffle=False, pin_memory=True)

In [ ]:
# dataset1 : Display image and label chips.
train_features, train_labels = next(iter(train_set1_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
print(train_features.dtype)
print(train_labels.dtype)

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
rows = len(train_features)
cols = (train_features.shape[1]+train_labels.shape[1])
fig, axs = plt.subplots(nrows=rows, ncols=cols, layout='constrained',figsize=(5,30))
for i in range(rows):
    axs[i,0].imshow(train_features[i][0],cmap='gray')
    axs[i,0].set_title('VV', fontsize=10)
    axs[i,1].imshow(train_features[i][1],cmap='gray')
    axs[i,1].set_title('VH', fontsize=10)
    axs[i,2].imshow(train_features[i][2],cmap='gray')
    axs[i,2].set_title('dem', fontsize=10),
    axs[i,3].imshow(train_features[i][3],cmap='gray')
    axs[i,3].set_title('pwater', fontsize=10) ,
    axs[i,4].imshow(train_labels[i][0],cmap='Blues')
    axs[i,4].set_title('Water_label', fontsize=10)   
plt.show()

In [ ]:
#from torchsummary import summary
#summary(model, (4, 256, 256))

In [ ]:
def train(dataloader, model, loss_fn, optimizer,tb_writer,epoch_number):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    last_loss,running_loss = 0.0,0.0
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        # Valid mask
        valid_pixel_mask = y.ne(255)
        pred = pred.masked_select(valid_pixel_mask)
        y = y.masked_select(valid_pixel_mask)
        loss = loss_fn(pred, y)
        running_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    last_loss = running_loss/num_batches
    print(f"Train    : Avg loss: {last_loss:>8f} \n")
    tb_writer.add_scalar('Loss/train', last_loss, epoch_number)

    return last_loss

In [ ]:
def IoU(pred, true):
    valid_pixel_mask = true.ne(255)
    pred_arr = pred.masked_select(valid_pixel_mask).to("cpu")
    pred_arr = pred_arr.numpy()
    pred_arr = np.where(pred_arr>0.5,1,0)
    true_arr = true.masked_select(valid_pixel_mask).to("cpu")
    true_arr = true_arr.numpy()
    # Intersection and union totals
    intersection = np.logical_and(true_arr, pred_arr)
    union = np.logical_or(true_arr, pred_arr)   
    iou = intersection.sum()/(union.sum()+0.0000001)
    return iou

In [ ]:
def test(dataloader, model, loss_fn,tb_writer,epoch_number):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    last_loss, last_iou, running_loss, running_iou = 0, 0, 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            # Valid mask
            valid_pixel_mask = y.ne(255)
            pred = pred.masked_select(valid_pixel_mask)
            y = y.masked_select(valid_pixel_mask)
            loss = loss_fn(pred, y)
            running_loss += loss.item()
            iou = IoU(pred, y)
            running_iou += iou

    last_loss = running_loss/num_batches
    last_iou = running_iou/num_batches

    print(f"Test     : \n Accuracy IOU: {(last_iou):>8f}%, Avg loss: {last_loss:>8f} \n")
    tb_writer.add_scalar('Loss/test', last_loss, epoch_number)
    tb_writer.add_scalar('Accuracy/IOU', last_iou, epoch_number)
    
    return last_loss

Model training

In [ ]:
# Training on GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Model training on dataset 01
model = Unet(4,1).to(device)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/s1_unet_{}'.format(timestamp))

epochs = 50
best_vloss = 1_000_000.
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_avg_loss = train(train_set1_dataloader, model, loss_fn, optimizer,writer,t+1)
    test_avg_loss = test(validation_set1_dataloader, model, loss_fn,writer,t+1)
    
    writer.add_scalars('Train vs. Validation Loss',
                    { 'Training' : train_avg_loss, 'Validation' : test_avg_loss },t + 1)
    writer.flush()
    
    if test_avg_loss < best_vloss:
        best_vloss = test_avg_loss
        model_path = './models/model_{}_{}'.format(timestamp, t+1)
        torch.save(model.state_dict(), model_path)
        
torch.cuda.empty_cache()
print("Done!")

In [ ]:
# Model training on dataset 02
model = Unet(4,1).to(device)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/s1_unet_{}'.format(timestamp))

epochs = 100
best_vloss = 1_000_000.
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_avg_loss = train(train_set2_dataloader, model, loss_fn, optimizer,writer,t+1)
    test_avg_loss = test(validation_set2_dataloader, model, loss_fn,writer,t+1)
    
    writer.add_scalars('Train vs. Validation Loss',
                    { 'Training' : train_avg_loss, 'Validation' : test_avg_loss },t + 1)
    writer.flush()
    
    if test_avg_loss < best_vloss:
        best_vloss = test_avg_loss
        model_path = './models/model_{}_{}'.format(timestamp, t+1)
        torch.save(model.state_dict(), model_path)
torch.cuda.empty_cache()
print("Done!")